In [ ]:
from google.colab import files
files.upload()  # Upload your 'kaggle.json' file here


{}

#Downloading the dataset directly from kaggle to drive

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!kaggle datasets download -d joe1995/div2k-dataset -p "/content/drive/MyDrive/Digital Image Processing_Major Project"


Dataset URL: https://www.kaggle.com/datasets/joe1995/div2k-dataset
License(s): unknown
100% 3.70G/3.71G [00:50<00:00, 74.2MB/s]
100% 3.71G/3.71G [00:50<00:00, 78.1MB/s]


In [ ]:
!unzip "/content/drive/MyDrive/Digital Image Processing_Major Project/div2k-dataset.zip" -d "/content/drive/MyDrive/Digital Image Processing_Major Project/"



Archive:  /content/drive/MyDrive/Digital Image Processing_Major Project/div2k-dataset.zip
  inflating: /content/drive/MyDrive/Digital Image Processing_Major Project/DIV2K_train_HR/DIV2K_train_HR/0001.png  
  inflating: /content/drive/MyDrive/Digital Image Processing_Major Project/DIV2K_train_HR/DIV2K_train_HR/0002.png  
  inflating: /content/drive/MyDrive/Digital Image Processing_Major Project/DIV2K_train_HR/DIV2K_train_HR/0003.png  
  inflating: /content/drive/MyDrive/Digital Image Processing_Major Project/DIV2K_train_HR/DIV2K_train_HR/0004.png  
  inflating: /content/drive/MyDrive/Digital Image Processing_Major Project/DIV2K_train_HR/DIV2K_train_HR/0005.png  
  inflating: /content/drive/MyDrive/Digital Image Processing_Major Project/DIV2K_train_HR/DIV2K_train_HR/0006.png  
  inflating: /content/drive/MyDrive/Digital Image Processing_Major Project/DIV2K_train_HR/DIV2K_train_HR/0007.png  
  inflating: /content/drive/MyDrive/Digital Image Processing_Major Project/DIV2K_train_HR/DIV2K_tr

#Data Preparation

In [ ]:
import os
import shutil
import random

In [ ]:
def split_dataset(input_dir, train_dir, val_dir, test_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    # Ensure the ratios sum to 1
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1"

    # Create output directories
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get all files and shuffle
    all_files = os.listdir(input_dir)
    random.shuffle(all_files)

    # Calculate split sizes
    total_files = len(all_files)
    train_size = int(total_files * train_ratio)
    val_size = int(total_files * val_ratio)

    # Split the data
    train_files = all_files[:train_size]
    val_files = all_files[train_size:train_size + val_size]
    test_files = all_files[train_size + val_size:]

    # Copy files to respective directories
    for file in train_files:
        shutil.copy(os.path.join(input_dir, file), os.path.join(train_dir, file))
    for file in val_files:
        shutil.copy(os.path.join(input_dir, file), os.path.join(val_dir, file))
    for file in test_files:
        shutil.copy(os.path.join(input_dir, file), os.path.join(test_dir, file))

    print(f"Dataset split complete:")
    print(f"  Train: {len(train_files)} files")
    print(f"  Validate: {len(val_files)} files")
    print(f"  Test: {len(test_files)} files")

# Directories
input_dir = "/content/drive/MyDrive/Digital Image Processing_Major Project/DIV2K_train_HR/DIV2K_train_HR"
train_dir = "/content/drive/MyDrive/Digital Image Processing_Major Project/Split_Dataset/train"
val_dir = "/content/drive/MyDrive/Digital Image Processing_Major Project/Split_Dataset/validate"
test_dir = "/content/drive/MyDrive/Digital Image Processing_Major Project/Split_Dataset/test"

# Split and create folders
split_dataset(input_dir, train_dir, val_dir, test_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1)


Dataset split complete:
  Train: 560 files
  Validate: 160 files
  Test: 80 files


In [ ]:
from PIL import Image, ImageFilter
import numpy as np
import os


In [ ]:
def generate_lr_images_with_degradations(input_dir, output_dir, scale_factor=4, blur_radius=2, noise_std=10):
    """
    Generate Low-Resolution (LR) images with added blur and noise.

    Args:
        input_dir (str): Path to the directory containing High-Resolution (HR) images.
        output_dir (str): Path to save the degraded Low-Resolution (LR) images.
        scale_factor (int): Downscaling factor.
        blur_radius (int): Radius for Gaussian blur.
        noise_std (float): Standard deviation for Gaussian noise.
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Loop through all images in the input directory
    for img_name in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_name)

        try:
            # Open the image
            img = Image.open(img_path)

            # Downscale the image
            new_width = img.width // scale_factor
            new_height = img.height // scale_factor
            lr_img = img.resize((new_width, new_height), Image.BICUBIC)

            # Apply Gaussian blur
            lr_img = lr_img.filter(ImageFilter.GaussianBlur(blur_radius))

            # Add Gaussian noise
            lr_img_array = np.array(lr_img)
            noise = np.random.normal(0, noise_std, lr_img_array.shape).astype(np.int16)
            noisy_img_array = np.clip(lr_img_array + noise, 0, 255).astype(np.uint8)
            noisy_img = Image.fromarray(noisy_img_array)

            # Save the LR image to the output directory
            noisy_img.save(os.path.join(output_dir, img_name))
            print(f"Generated degraded LR image for {img_name}")

        except Exception as e:
            print(f"Error processing {img_name}: {e}")


In [ ]:
# Paths for HR and LR directories
base_dir = "/content/drive/MyDrive/Digital Image Processing_Major Project/Split_Dataset"

train_hr_dir = os.path.join(base_dir, "train")
train_lr_dir = os.path.join(base_dir, "train_LR_degraded")

validate_hr_dir = os.path.join(base_dir, "validate")
validate_lr_dir = os.path.join(base_dir, "validate_LR_degraded")

test_hr_dir = os.path.join(base_dir, "test")
test_lr_dir = os.path.join(base_dir, "test_LR_degraded")


In [ ]:
print("Generating degraded LR images for training data...")
generate_lr_images_with_degradations(train_hr_dir, train_lr_dir, scale_factor=4, blur_radius=2, noise_std=10)


Generating degraded LR images for training data...
Generated degraded LR image for 0798.png
Generated degraded LR image for 0055.png
Generated degraded LR image for 0794.png
Generated degraded LR image for 0616.png
Generated degraded LR image for 0681.png
Generated degraded LR image for 0588.png
Generated degraded LR image for 0388.png
Generated degraded LR image for 0325.png
Generated degraded LR image for 0675.png
Generated degraded LR image for 0700.png
Generated degraded LR image for 0530.png
Generated degraded LR image for 0655.png
Generated degraded LR image for 0781.png
Generated degraded LR image for 0557.png
Generated degraded LR image for 0509.png
Generated degraded LR image for 0464.png
Generated degraded LR image for 0676.png
Generated degraded LR image for 0082.png
Generated degraded LR image for 0521.png
Generated degraded LR image for 0248.png
Generated degraded LR image for 0380.png
Generated degraded LR image for 0272.png
Generated degraded LR image for 0543.png
Genera

In [ ]:
print("Generating degraded LR images for validation data...")
generate_lr_images_with_degradations(validate_hr_dir, validate_lr_dir, scale_factor=4, blur_radius=2, noise_std=10)


Generating degraded LR images for validation data...
Generated degraded LR image for 0389.png
Generated degraded LR image for 0465.png
Generated degraded LR image for 0222.png
Generated degraded LR image for 0183.png
Generated degraded LR image for 0649.png
Generated degraded LR image for 0035.png
Generated degraded LR image for 0254.png
Generated degraded LR image for 0289.png
Generated degraded LR image for 0239.png
Generated degraded LR image for 0505.png
Generated degraded LR image for 0583.png
Generated degraded LR image for 0312.png
Generated degraded LR image for 0058.png
Generated degraded LR image for 0783.png
Generated degraded LR image for 0515.png
Generated degraded LR image for 0444.png
Generated degraded LR image for 0410.png
Generated degraded LR image for 0156.png
Generated degraded LR image for 0369.png
Generated degraded LR image for 0296.png
Generated degraded LR image for 0712.png
Generated degraded LR image for 0564.png
Generated degraded LR image for 0553.png
Gene

In [ ]:
print("Generating degraded LR images for testing data...")
generate_lr_images_with_degradations(test_hr_dir, test_lr_dir, scale_factor=4, blur_radius=2, noise_std=10)


Generating degraded LR images for testing data...
Generated degraded LR image for 0413.png
Generated degraded LR image for 0768.png
Generated degraded LR image for 0306.png
Generated degraded LR image for 0703.png
Generated degraded LR image for 0346.png
Generated degraded LR image for 0339.png
Generated degraded LR image for 0205.png
Generated degraded LR image for 0584.png
Generated degraded LR image for 0526.png
Generated degraded LR image for 0390.png
Generated degraded LR image for 0462.png
Generated degraded LR image for 0334.png
Generated degraded LR image for 0685.png
Generated degraded LR image for 0623.png
Generated degraded LR image for 0672.png
Generated degraded LR image for 0317.png
Generated degraded LR image for 0157.png
Generated degraded LR image for 0364.png
Generated degraded LR image for 0503.png
Generated degraded LR image for 0197.png
Generated degraded LR image for 0724.png
Generated degraded LR image for 0366.png
Generated degraded LR image for 0586.png
Generat